In [ ]:
# Setup

import os
from dotenv import load_dotenv

from gemini_batcher.gemini_batcher import GeminiBatcher
from gemini_batcher.input_handler.text_inputs import WebsiteInput
from gemini_batcher.input_handler.media_inputs import VideoFileInput

from gemini_batcher.gemini_config import GeminiConfig
from gemini_batcher.strategies import *

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [2]:
# Generating the content and questions to ask about.
# The content was gathered from a MIT lecture transcript which is accessible at https://ocw.mit.edu/courses/6-00-introduction-to-computer-science-and-programming-fall-2008/
# The questions were generated by Gemini using the transcript.

import httpx

questions = httpx.get("https://raw.githubusercontent.com/phil-daniel/gemini-batcher/refs/heads/main/examples/demo_files/questions.txt").text.split('\n')
content = WebsiteInput("https://raw.githubusercontent.com/phil-daniel/gemini-batcher/refs/heads/main/examples/demo_files/content.txt")

In [ ]:
# Setting the configuration of Gemini, these can be changed in the future.

config = GeminiConfig(
    api_key=GEMINI_API_KEY,
    model="gemini-2.0-flash",
)

# Creating the Gemini Batcher object.
client = GeminiBatcher(config)

# Setting the strategies you wish to use.
chunking_strategy = TextSlidingWindowChunking(
    chunk_char_size=30000,
    window_char_size=0
)

# Since the batch_size > len(batch) only 1 batch is used.
batching_strategy = FixedBatching(
    batch_size=50
)

In [ ]:
# Generates a list of answers to the inputted questions using the information in the content provided.
answers = client.generate_content(
    content=content,
    questions=questions,
    chunking_strategy=chunking_strategy,
    batching_strategy=batching_strategy,
)

print(f'Question 1: {questions[0]}')
print(f'Answer 1: {answers.content[questions[0]]}')
    

Question 1: What is the goal of MIT 6.00 (Introduction to Computer Science and Programming)?
Answer 1: To help everybody learn about computation.


Batching

In [5]:
# We can compare the number of tokens required when batching a different number of questions with one another.

chunking_strategy = TextSlidingWindowChunking(
    chunk_char_size=100000,
    window_char_size=0
)

for i in [1, 2, 5, 15, 30]:
    print(f'Batch size: {i}')
    batching_strategy = FixedBatching(batch_size=i)
    answers = client.generate_content(
        content=content,
        questions=questions,
        chunking_strategy=chunking_strategy,
        batching_strategy=batching_strategy,
        config=config
    )
    print(f'\tInput tokens: {answers.input_tokens}, Output tokens: {answers.output_tokens}')

Batch size: 1


	Input tokens: 401777, Output tokens: 956
Batch size: 2


	Input tokens: 201092, Output tokens: 1038
Batch size: 5
	Input tokens: 80681, Output tokens: 855
Batch size: 15
	Input tokens: 27167, Output tokens: 839
Batch size: 30
	Input tokens: 13787, Output tokens: 781


In [ ]:
# Semantic Chunking

res = client.generate_content_semantic(content, questions)

print (res.content)
print (res.input_tokens)
print (res.output_tokens)

{'What is the goal of MIT 6.00 (Introduction to Computer Science and Programming)?': 'N/A', 'Who teaches the course?': 'Eric Grimson and John Guttag teach the course.', 'What programming language is used in the course?': 'N/A', 'What are the strategic and tactical goals of the course?': "The strategic goals are to help prepare freshmen and sophomores interested in majoring in course six to get an easy entry into the department, especially those without prior programming experience. Also, to help students who don't plan to major in course six to feel confident in their ability to write and read small pieces of code. For all students, the goal is to provide an understanding of the role computation can and cannot play in tackling technical problems and to position all students to compete for things like office and summer jobs.", 'What kind of students is this course intended for?': 'This course is primarily aimed at students who have little or no prior programming experience.', 'How are q

In [ ]:
# client.generate_content_media("./short.mp4", [])

client.generate_content_media(VideoFileInput("short.mp4"), [])



ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

ACTIVE
["I think I've always liked making things from a very young age. Um, I remember when I was very young I used to tell people that I wanted to be an inventor. I decided I wanted to study computer science at university, um, part way through my computer science AS level. But it was probably my uncle who used to make websites that got me interested in computing when I was maybe ten or 11 years old. I'm glad that I didn't choose computer science when I went to university because, you know, it's been transformed, um, in the couple of decades since then into a subject that's really worthwhile studying that has a much deeper, um, theoretical basis that it is worth spending three or four years of your time studying. I love solving problems, and I love finding creative solutions.", "The thing that sets Oxford apart from most universities is that it focuses quite a lot on the theoretical aspects of computer science and that's what drew me to it. The, uh, heart of our computer science course

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
